In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys

from dotenv import load_dotenv

load_dotenv("../config/prod.env")

import yaml
with open("../config/config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

import pandas as pd
import mlflow

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.entities import (
    BatchEndpoint,
    ModelBatchDeployment,
    ModelBatchDeploymentSettings,
    Data,
    BatchRetrySettings,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction

from datetime import datetime


# enter details of your AML workspace
subscription_id = os.getenv("subscription_id")
resource_group = os.getenv("resource_group")
workspace = os.getenv("workspace")

credential = InteractiveBrowserCredential() #DefaultAzureCredential()
# get a handle to the workspace
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

# Create Batch Endpoint

Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use BatchEndpoint. This class allows user to configure the following key aspects:

name - Name of the endpoint. Needs to be unique at the Azure region level
auth_mode - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (aad_token) authentication is supported.
description- Description of the endpoint.

In [ ]:
endpoint_name = cfg["batch_endpoint"]["name"]

# # Get the current time
# current_time = datetime.now()
# # Convert the current time to a string using the strftime method
# time_string = current_time.strftime("%Y%m%d%H%M%S")
# endpoint_name = f"{endpoint_name}-{time_string}"

print(f"Endpoint name: {endpoint_name}")

In [ ]:
endpoint = BatchEndpoint(
    name=endpoint_name, #cfg["endpoint"]["name"],
    description=cfg["batch_endpoint"]["description"],
)
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()


# Create a deployment

In [ ]:
deployment = ModelBatchDeployment(
    name=cfg["batch_endpoint"]["deployment"]["name"],
    description=cfg["batch_endpoint"]["deployment"]["description"],
    endpoint_name=cfg["batch_endpoint"]["name"],
    model=ml_client.models.get(name=cfg["model"]["name"], version=cfg["model"]["version"]),
    code_configuration=CodeConfiguration(
        code="../src/", scoring_script="score_batch.py"
    ),
    environment=ml_client.environments.get(name=cfg["batch_endpoint"]["environment"]["name"], version=cfg["batch_endpoint"]["environment"]["version"]),
    compute=cfg["batch_endpoint"]["compute"]["name"],
    settings=ModelBatchDeploymentSettings(
        instance_count=cfg["batch_endpoint"]["deployment"]["instance_count"],
        max_concurrency_per_instance=cfg["batch_endpoint"]["deployment"]["max_concurrency_per_instance"],
        mini_batch_size=cfg["batch_endpoint"]["deployment"]["mini_batch_size"],
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name=cfg["batch_endpoint"]["deployment"]["output_file_name"],
        retry_settings=BatchRetrySettings(max_retries=1, timeout=300),
        logging_level=cfg["batch_endpoint"]["deployment"]["logging_level"],
    ),
    
)

In [ ]:
ml_client.begin_create_or_update(deployment).result()


In [ ]:
ml_client.batch_deployments.get(name=deployment.name, endpoint_name=endpoint.name)


# Test invoke

In [ ]:
invocation_data = Data(
    path=cfg["data"]["invocation"]["path"],
    type=AssetTypes.URI_FOLDER,
    description=cfg["data"]["invocation"]["description"],
    name=cfg["data"]["invocation"]["name"],
)
ml_client.data.create_or_update(invocation_data)


In [ ]:
invocation_data = ml_client.data.get(name=cfg["data"]["invocation"]["name"], label=cfg["data"]["invocation"]["version"])

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=cfg["batch_endpoint"]["name"],
    deployment_name=cfg["batch_endpoint"]["deployment"]["name"],
    input=Input(
        path=invocation_data.id,
        type=AssetTypes.URI_FOLDER,
    ),
)

In [ ]:
# az ml batch-endpoint invoke --name rat-classifier --deployment-name rat-deployment --input data/endpoint/sample_invocation/ --instance-count 1 --mini-batch-size 2 --resource-group my-resource-group --workspace-name my-workspace


## Control Output location

In [ ]:
datastore = ml_client.datastores.get(name=cfg["datastore"]["name"])
datastore.id

In [ ]:
data_path = ""

In [ ]:
job = ml_client.batch_endpoints.invoke(
   endpoint_name=cfg["batch_endpoint"]["name"],
   deployment_name=cfg["batch_endpoint"]["deployment"]["name"],
   input=Input(
      path=invocation_data.id,
      type=AssetTypes.URI_FOLDER,
   ),
   outputs=Output(
      type=AssetTypes.URI_FOLDER, 
      path=f"{datastore.id}/paths/{data_path}"
   )
)